# Experimentos Finales Modelos no RN

In [32]:
import pandas as pd
import numpy as np

import missingno as msno
import matplotlib.pyplot as plt
import plotly.express as px

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split 
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.decomposition import TruncatedSVD

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
import datetime
from dateutil.parser import parse
from sklearn.tree import  DecisionTreeClassifier
import plotly.graph_objects as go
import umap
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler
import xgboost as xgb
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import GridSearchCV
import ml_metrics
import pickle
from sklearn.metrics import accuracy_score
import json


In [33]:
Data=pd.read_csv("cleaned_01June2022.csv").drop(columns=["Unnamed: 0"]).fillna("")
Data.head()

c:\Users\Jose Luis\anaconda3\envs\Programacion_Cientifica\lib\site-packages\IPython\core\interactiveshell.py:3441: DtypeWarning: Columns (40) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,pref_B-B,pref_D-E,pref_C-D,pref_A-A,Duracion_Campaña,B,C,lec_B-B,lec_E-E,lec_D-E,...,Renta,Recibe_sueldo_en_cuenta,Segmento_consumidor,Meses_antiguedad,Comuna,Ciudad,Estado_civil,Principalidad,Profesion,Target
0,0.0,0.0,0.0,0.0,1.0,4.0,5.0,5.0,0.0,0.0,...,R1,0,A,Mayor a 10 años,331.0,13.0,D,B,P164,
1,0.0,0.0,0.0,0.0,1.0,2.0,9.0,14.0,1.0,1.0,...,R9,0,B,Mayor a 10 años,37.0,5.0,D,E,P164,D-E C-D E-E B-B
2,0.0,0.0,0.0,0.0,1.0,2.0,2.0,5.0,0.0,0.0,...,R9,1,B,Mayor a 10 años,19.0,4.0,B,D,P114,E-E A-A B-B
3,0.0,0.0,0.0,0.0,1.0,6.0,6.0,8.0,0.0,1.0,...,R4,0,A,Mayor a 10 años,91.0,13.0,D,B,P85,E-E B-B D-E
4,0.0,0.0,0.0,0.0,1.0,2.0,3.0,1.0,0.0,0.0,...,R5,0,A,Mayor a 10 años,34.0,5.0,B,C,P143,D-E A-A


In [34]:
# datos númericos
numerical=list(Data.select_dtypes(exclude=["category","object"]).columns)
numerical.remove("Sexo")
numerical.remove('Recibe_sueldo_en_cuenta')


# datos categorical
categorical=list(Data.select_dtypes(include=["category","object"]).columns)+['Sexo','Recibe_sueldo_en_cuenta']
categorical.remove("Target")

In [35]:
for i in categorical:
    Data[i]=Data[i].astype(str)

In [36]:
# Select 1 - Only Gini, corte visual
select_variables_1=['E-F',
 'C-C',
 'D-F',
 'A-K',
 'num_comunicaciones',
 'Length',
 'pref_A-A',
 'Positivo',
 'Recency',
 'A-G',
 'Monto',
 'Periodicity',
 'Frequency',
 'lec_B-B',
 'G-K']

In [37]:
# Select 2 - Criterio Corr Kill
select_variables_2=['E-F',
 'C-C',
 'D-F',
 'A-K',
 'pref_A-A',
 'num_comunicaciones',
 'Length',
 'Sin Info',
 'F-I',
 'Monto',
 'B',
 'lec_B-B',
 'Comuna',
 'Edad',
 'Ciudad',
 'Profesion',
 'Renta',
 'F-H',
 'lec_A-A',
 'Principalidad',
 'A-L',
 'lec_C-D',
 'Estado_civil']

# Train, Test Split

In [38]:
X=Data.drop(columns=["Target"]).copy()
X.head()

,pref_B-B,pref_D-E,pref_C-D,pref_A-A,Duracion_Campaña,B,C,lec_B-B,lec_E-E,lec_D-E,...,Sexo,Renta,Recibe_sueldo_en_cuenta,Segmento_consumidor,Meses_antiguedad,Comuna,Ciudad,Estado_civil,Principalidad,Profesion
0,0.0,0.0,0.0,0.0,1.0,4.0,5.0,5.0,0.0,0.0,...,1,R1,0,A,Mayor a 10 años,331.0,13.0,D,B,P164
1,0.0,0.0,0.0,0.0,1.0,2.0,9.0,14.0,1.0,1.0,...,1,R9,0,B,Mayor a 10 años,37.0,5.0,D,E,P164
2,0.0,0.0,0.0,0.0,1.0,2.0,2.0,5.0,0.0,0.0,...,1,R9,1,B,Mayor a 10 años,19.0,4.0,B,D,P114
3,0.0,0.0,0.0,0.0,1.0,6.0,6.0,8.0,0.0,1.0,...,1,R4,0,A,Mayor a 10 años,91.0,13.0,D,B,P85
4,0.0,0.0,0.0,0.0,1.0,2.0,3.0,1.0,0.0,0.0,...,1,R5,0,A,Mayor a 10 años,34.0,5.0,B,C,P143


In [39]:
Y=Data["Target"]
Y.head()

0                   
1    D-E C-D E-E B-B
2        E-E A-A B-B
3        E-E B-B D-E
4            D-E A-A
Name: Target, dtype: object

-----------

dic_target={'': 0,
 'A-A': 36,
 'A-A B-B': 15,
 'A-A B-B D-E': 43,
 'A-A C-D': 73,
 'A-A D-E': 44,
 'A-A D-E B-B': 50,
 'A-A E-E': 39,
 'B-B': 18,
 'B-B A-A': 35,
 'B-B A-A D-E': 42,
 'B-B C-D': 59,
 'B-B C-D D-E': 37,
 'B-B C-D D-E E-E': 64,
 'B-B C-D E-E': 62,
 'B-B D-E': 13,
 'B-B D-E A-A': 71,
 'B-B D-E C-D': 60,
 'B-B D-E E-E': 29,
 'B-B E-E': 9,
 'B-B E-E C-D': 65,
 'B-B E-E D-E': 34,
 'C-D': 21,
 'C-D A-A': 54,
 'C-D A-A B-B': 25,
 'C-D B-B': 20,
 'C-D B-B A-A': 61,
 'C-D B-B D-E': 16,
 'C-D B-B D-E E-E': 68,
 'C-D B-B E-E': 12,
 'C-D B-B E-E D-E': 14,
 'C-D D-E': 31,
 'C-D D-E B-B': 24,
 'C-D D-E B-B E-E': 56,
 'C-D D-E E-E': 23,
 'C-D D-E E-E B-B': 70,
 'C-D E-E': 19,
 'C-D E-E B-B': 32,
 'C-D E-E B-B D-E': 38,
 'C-D E-E D-E': 28,
 'C-D E-E D-E B-B': 33,
 'D-E': 6,
 'D-E A-A': 4,
 'D-E A-A B-B': 41,
 'D-E B-B': 7,
 'D-E B-B A-A': 8,
 'D-E B-B C-D': 63,
 'D-E B-B E-E': 46,
 'D-E C-D': 45,
 'D-E C-D B-B': 40,
 'D-E C-D B-B E-E': 66,
 'D-E C-D E-E': 67,
 'D-E C-D E-E B-B': 1,
 'D-E E-E': 22,
 'D-E E-E A-A': 72,
 'D-E E-E B-B': 10,
 'D-E E-E C-D': 11,
 'E-E': 5,
 'E-E A-A': 57,
 'E-E A-A B-B': 2,
 'E-E A-A D-E': 48,
 'E-E B-B': 17,
 'E-E B-B A-A': 69,
 'E-E B-B C-D': 55,
 'E-E B-B D-E': 3,
 'E-E C-D': 49,
 'E-E C-D B-B': 52,
 'E-E C-D B-B D-E': 58,
 'E-E C-D D-E': 53,
 'E-E C-D D-E B-B': 47,
 'E-E D-E': 26,
 'E-E D-E A-A': 51,
 'E-E D-E B-B': 30,
 'E-E D-E C-D': 27}

In [40]:

#tf = open("myDictionary.json", "w")
#json.dump(dic_target,tf)
#tf.close()

In [41]:
tf = open("myDictionary.json", "r")
dic_target = json.load(tf)

In [42]:
# Replace solo para experimentos con XGboost
Y.replace(dic_target,inplace=True)

-----------------

In [43]:
# Generación de conjuntos de train y test
Xtrain, Xtest, Ytrain, Ytest = train_test_split(
    X, Y, test_size=0.33, 
    shuffle=True,
    
) 

In [44]:
# Sobremuestreo: Aumentar numero de datos de la clase minoritaria
# ROS(Duplica clases):
ros=RandomOverSampler(random_state=0) # Random_state=0

In [45]:
# ROS:
Xtrain, Ytrain=ros.fit_resample(Xtrain,Ytrain)

In [46]:
Xtrain.shape

(613608, 43)

In [47]:
numerical1=list(filter(lambda x:x  in numerical,select_variables_1))
categorical1=list(filter(lambda x:x  in categorical,select_variables_1))

In [48]:
numerical2=list(filter(lambda x:x  in numerical,select_variables_2))
categorical2=list(filter(lambda x:x  in categorical,select_variables_2))

In [49]:
# Preprocesamiento de datos
preprocessing_transformer1 = ColumnTransformer(
    transformers=[('OneHotEncoder', OneHotEncoder(handle_unknown="ignore"),categorical1 ),
        ('MinMax', MinMaxScaler(),numerical1),
    ])

In [50]:
# Preprocesamiento de datos
preprocessing_transformer2 = ColumnTransformer(
    transformers=[('OneHotEncoder', OneHotEncoder(handle_unknown="ignore"),categorical2 ),
        ('MinMax', MinMaxScaler(),numerical2),
    ])

# Exp 1

In [107]:
pipe_naive1 = Pipeline(
    [
        ("preprocesamiento", preprocessing_transformer1), 
     ("clf",MultinomialNB())
    ]
)

In [108]:
# Fit
pipe_naive1.fit(Xtrain, Ytrain)

Pipeline(steps=[('preprocesamiento',
                 ColumnTransformer(transformers=[('OneHotEncoder',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  []),
                                                 ('MinMax', MinMaxScaler(),
                                                  ['E-F', 'C-C', 'D-F', 'A-K',
                                                   'num_comunicaciones',
                                                   'Length', 'pref_A-A',
                                                   'Positivo', 'Recency', 'A-G',
                                                   'Monto', 'Periodicity',
                                                   'Frequency', 'lec_B-B',
                                                   'G-K'])])),
                ('clf', MultinomialNB())])

In [21]:
# Generación de predicciones
Ypred = pipe_naive1.predict(Xtest)

In [22]:
accuracy_score(Ypred, Ytest)

0.32540593734623585

In [23]:
Ypred_map=pd.Series(Ypred).apply( lambda x: x.split(" "))
Ytest_map=pd.Series(Ytest).apply( lambda x: x.split(" "))
ml_metrics.mapk(Ytest_map, Ypred_map, 5)

0.7002533121936106

In [112]:
# Guardar modelo:
pickle.dump(pipe_naive1, open('pipe_naive1.pkl', "wb"))

In [17]:
# Leer modelo:
pipe_naive1= pickle.load(open('pipe_naive1.pkl', 'rb'))

# Exp 2

In [113]:
pipe_naive2 = Pipeline(
    [
        ("preprocesamiento", preprocessing_transformer2), 
     ("clf",MultinomialNB())
    ]
)

In [114]:
# Fit
pipe_naive2.fit(Xtrain, Ytrain)

Pipeline(steps=[('preprocesamiento',
                 ColumnTransformer(transformers=[('OneHotEncoder',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['Comuna', 'Edad', 'Ciudad',
                                                   'Profesion', 'Renta',
                                                   'Principalidad',
                                                   'Estado_civil']),
                                                 ('MinMax', MinMaxScaler(),
                                                  ['E-F', 'C-C', 'D-F', 'A-K',
                                                   'pref_A-A',
                                                   'num_comunicaciones',
                                                   'Length', 'Sin Info', 'F-I',
                                                   'Monto', 'B', 'lec_B-B',
                                                   'F-H', 'lec_A-A', '

In [115]:
# Generación de predicciones
Ypred = pipe_naive2.predict(Xtest)

In [116]:
accuracy_score(Ypred, Ytest)

0.07831720518287683

In [117]:
Ypred_map=pd.Series(Ypred).apply( lambda x: x.split(" "))
Ytest_map=pd.Series(Ytest).apply( lambda x: x.split(" "))
ml_metrics.mapk(Ytest_map, Ypred_map, 5)

0.5296001676598691

**Modelo descartado**

# Exp 3

In [20]:
pipe_random_forest1= Pipeline(
    [
        ("preprocesamiento", preprocessing_transformer1), 
     ("clf",RandomForestClassifier(random_state=1,n_estimators=100))
    ]
)

In [21]:
# Fit
pipe_random_forest1.fit(Xtrain, Ytrain)

Pipeline(steps=[('preprocesamiento',
                 ColumnTransformer(transformers=[('OneHotEncoder',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  []),
                                                 ('MinMax', MinMaxScaler(),
                                                  ['E-F', 'C-C', 'D-F', 'A-K',
                                                   'num_comunicaciones',
                                                   'Length', 'pref_A-A',
                                                   'Positivo', 'Recency', 'A-G',
                                                   'Monto', 'Periodicity',
                                                   'Frequency', 'lec_B-B',
                                                   'G-K'])])),
                ('clf', RandomForestClassifier(random_state=1))])

In [25]:
# Generación de predicciones
Ypred = pipe_random_forest1.predict(Xtest)

In [26]:
accuracy_score(Ypred, Ytest)

0.8009266852550435

In [27]:
Ypred_map=pd.Series(Ypred).apply( lambda x: x.split(" "))
Ytest_map=pd.Series(Ytest).apply( lambda x: x.split(" "))
ml_metrics.mapk(Ytest_map, Ypred_map, 5)

0.908326954968746

In [36]:
# Guardar modelo:
pickle.dump(pipe_random_forest1, open('pipe_random_forest1.pkl', "wb"))

In [24]:
# Leer modelo:
pipe_random_forest1= pickle.load(open('pipe_random_forest1.pkl', 'rb'))

# Exp 4

In [37]:
pipe_random_forest2 = Pipeline(
    [
        ("preprocesamiento", preprocessing_transformer2), 
     ("clf",RandomForestClassifier(random_state=1,n_estimators=100))
    ]
)

In [38]:
# Fit
pipe_random_forest2.fit(Xtrain, Ytrain)

Pipeline(steps=[('preprocesamiento',
                 ColumnTransformer(transformers=[('OneHotEncoder',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['Comuna', 'Edad', 'Ciudad',
                                                   'Profesion', 'Renta',
                                                   'Principalidad',
                                                   'Estado_civil']),
                                                 ('MinMax', MinMaxScaler(),
                                                  ['E-F', 'C-C', 'D-F', 'A-K',
                                                   'pref_A-A',
                                                   'num_comunicaciones',
                                                   'Length', 'Sin Info', 'F-I',
                                                   'Monto', 'B', 'lec_B-B',
                                                   'F-H', 'lec_A-A', '

In [29]:
# Generación de predicciones
Ypred = pipe_random_forest2.predict(Xtest)

In [30]:
accuracy_score(Ypred, Ytest)

0.7951041495817616

In [31]:
Ypred_map=pd.Series(Ypred).apply( lambda x: x.split(" "))
Ytest_map=pd.Series(Ytest).apply( lambda x: x.split(" "))
ml_metrics.mapk(Ytest_map, Ypred_map, 5)

0.896110108796676

In [42]:
# Guardar modelo:
pickle.dump(pipe_random_forest2, open('pipe_random_forest2.pkl', "wb"))

In [28]:
# Leer modelo:
pipe_random_forest2= pickle.load(open('pipe_random_forest2.pkl', 'rb'))

# Modelos 5 fue entrenado en GPU de la nube de google

# Exp 5

In [ ]:
# save to BIN
#model_xgb1.save_model("./pipe_xgb1.bin")

In [105]:
# Load
model_xgb1 = xgb.XGBClassifier()
model_xgb1.load_model("model_xgb1.bin")

[18:51:32] WARNING: ..\src\learner.cc:854: Loading model from XGBoost < 1.0.0, consider saving it again for improved compatibility


c:\Users\Jose Luis\anaconda3\envs\Programacion_Cientifica\lib\site-packages\xgboost\sklearn.py:604: UserWarning: Loading a native XGBoost model with Scikit-Learn interface.
  warnings.warn(


In [106]:
# Leer pipeline:
pipe_xgb1= pickle.load(open('pipe_xgb1.pkl', 'rb'))

-----------

In [42]:
pipe_xgb1 = Pipeline(
    [("preprocesamiento", preprocessing_transformer1)]
)


In [ ]:
# Fit
pipe_xgb1.fit(Xtrain)

--------------

In [107]:
inv_map = {v: k for k, v in dic_target.items()}

In [108]:
Ypred=pd.DataFrame(model_xgb1.predict(pipe_xgb1.transform(Xtest))).replace(inv_map)[0]
Ypred

0        B-B D-E A-A
1        E-E B-B D-E
2            D-E B-B
3                A-A
4            A-A D-E
            ...     
24383    D-E E-E C-D
24384    B-B A-A D-E
24385        A-A E-E
24386    E-E B-B A-A
24387        D-E C-D
Name: 0, Length: 24388, dtype: object

In [109]:
Ytest.replace(inv_map,inplace=True)
Ytest

57554        D-E A-A
3375     B-B D-E E-E
26533        D-E B-B
42946            A-A
9048         B-B D-E
            ...     
5253     D-E E-E C-D
46082    A-A B-B D-E
58864        A-A E-E
28277    E-E A-A B-B
22994    C-D B-B D-E
Name: Target, Length: 24388, dtype: object

In [110]:
accuracy_score(Ypred, Ytest)

0.3574298835492865

In [111]:
Ypred_map=pd.Series(Ypred).apply( lambda x: x.split(" "))
Ytest_map=pd.Series(Ytest).apply( lambda x: x.split(" "))
ml_metrics.mapk(Ytest_map, Ypred_map, 5)

0.7315238140797842

In [52]:
# Guardar pipeline pre-process:
#pickle.dump(pipe_xgb1, open('pipe_xgb1.pkl', "wb"))